In [12]:
import torch
print(torch.__version__)             # Should be 2.4.0+cu127
print(torch.cuda.is_available())     # Should be True
print(torch.cuda.get_device_name(0)) # Should show NVIDIA GeForce MX330


2.2.2+cu121
True
NVIDIA GeForce MX330


In [5]:
from ultralytics import YOLO

# load the best model after training
model = YOLO("runs/train/weapons/train/weights/best.pt")

# run validation
metrics = model.val(data="D:/Project/SurakshaAI3/combined_gunsnknifes/data.yaml", imgsz=416, batch=2)

# print metrics
print("\n=== Final Metrics ===")
print(f"Precision:   {metrics.box.p.mean()*100:.2f}%")
print(f"Recall:      {metrics.box.r.mean()*100:.2f}%")
print(f"mAP@0.5:     {metrics.box.map50*100:.2f}%")
print(f"mAP@0.5:0.95 {metrics.box.map*100:.2f}%")


Ultralytics 8.3.0  Python-3.10.11 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce MX330, 2048MiB)


Model summary (fused): 186 layers, 2,684,758 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning D:\Project\SurakshaAI3\combined_gunsnknifes\val\labels.cache... 324 images, 17 backgrounds, 0 corrupt: 100%|██████████| 324/324 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 162/162 [00:05<00:00, 31.82it/s]


                   all        324        339      0.789      0.519      0.622       0.31
                pistol        190        222      0.757      0.369      0.472       0.24
                 knife        117        117      0.822      0.669      0.771      0.379
Speed: 0.6ms preprocess, 10.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\val2

=== Final Metrics ===
Precision:   78.92%
Recall:      51.93%
mAP@0.5:     62.16%
mAP@0.5:0.95 30.97%


In [8]:
# test_model.py
from ultralytics import YOLO

# Load the trained best model
model = YOLO("runs/train/weapons/train/weights/best.pt")

# Test on a single image
results = model.predict("im1.jpg", save=True, imgsz=416, conf=0.25)

# OR test on a folder of images
#results = model.predict(source="D:/Project/SurakshaAI3/combined_gunsnknifes/test", save=True, imgsz=416, conf=0.25)

# OR test on a video
# results = model.predict(source="test_video.mp4", save=True, imgsz=416, conf=0.25)


d:\Project\SurakshaAI3\.venv\lib\site-packages\ultralytics\utils\torch_utils.py:254: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:164.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
d:\Project\SurakshaAI3\.venv\lib\site-packages\ultralytics\utils\torch_utils.py:259: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, 

image 1/1 d:\Project\SurakshaAI3\im1.jpg: 416x416 3 knifes, 283.0ms
Speed: 2.0ms preprocess, 283.0ms inference, 27.1ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs\detect\predict3


In [ ]:
# continue_training.py
import torch
from ultralytics import YOLO

# GPU/CPU check
device = 0 if torch.cuda.is_available() else "cpu"
print(f"Training on: {'GPU' if device == 0 else 'CPU'}")

# load your last checkpoint (28 epochs)
model = YOLO("runs/train/weapons/train/weights/last.pt")

# continue training for +5 more epochs → total 33
model.train(
    data="D:/Project/SurakshaAI3/combined_gunsnknifes/data.yaml",
    epochs=50,
    batch=16 if torch.cuda.is_available() else 4,
    imgsz=640,
    project="runs/train",
    name="weapons_v2",   # ⚡ naya folder
    exist_ok=True,
    device=device,
)



In [ ]:
# validate_and_save.py
from ultralytics import YOLO
import shutil
import os

best_checkpoint = "runs/train/yolov8s_finetune_weapons/weights/best.pt"
model = YOLO(best_checkpoint)

metrics = model.val(data="D:/Project/SurakshaAI3/combined_gunsnknifes/data.yaml", imgsz=416, batch=2)
mAP50 = metrics.box.map50  # float between 0..1

print(f"mAP@0.5 = {mAP50*100:.2f}%")

if mAP50 >= 0.85:
    dst = "models/weapon_model_mAP85.pt"
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(best_checkpoint, dst)
    print(f"Saved model to {dst}")
else:
    print("mAP threshold not reached — not saving.")